In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from pmdarima.arima import auto_arima
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA,ARIMAResults
from sklearn.metrics import mean_squared_error,mean_absolute_percentage_error
from statsmodels.tools.eval_measures import rmse

from sklearn.preprocessing import OneHotEncoder
import pickle
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima.arima.utils import nsdiffs

In [36]:
#pip install --upgrade statsmodels

In [37]:
df_weekly= pd.read_csv('weekly_AQI (1).csv',index_col='DateTime',parse_dates=True)
df_weekly.head()

,id,AQI
DateTime,,
2021-06-06,ID001,49.448276
2021-06-13,ID001,159.000000
2021-06-20,ID001,159.000000
2021-06-27,ID001,159.000000
2021-07-04,ID001,159.000000


In [38]:
df_weekly.isna().sum()

id     0
AQI    0
dtype: int64

In [39]:
mean = df_weekly['AQI'].mean()
std_dev = df_weekly['AQI'].std()

# Calculate lower and upper bounds using Z-score method
lower_limit = mean - 3 * std_dev  
upper_limit = mean + 3 * std_dev 

# Identify outliers
outliers_before = df_weekly[(df_weekly['AQI'] < lower_limit) | (df_weekly['AQI'] > upper_limit)]

# Print outliers before ffill
print(f"Lower Limit: {lower_limit}")
print(f"Upper Limit: {upper_limit}")
print('No. of outliers before ffill:', len(outliers_before))
print(outliers_before)
print("\n")

# Forward fill outliers
df_weekly.loc[outliers_before.index, 'AQI'] = 0
df_weekly['AQI'] = df_weekly['AQI'].ffill()

# Identify outliers after ffill
outliers_after = df_weekly[(df_weekly['AQI'] < lower_limit) | (df_weekly['AQI'] > upper_limit)]

# Print outliers after ffill
print('No. of outliers after ffill:', len(outliers_after))
print(outliers_after)
print("\n")

Lower Limit: -148.25208495676367
Upper Limit: 376.6095452271358
No. of outliers before ffill: 40
               id         AQI
DateTime                     
2021-06-13  ID016  500.000000
2021-06-20  ID016  500.000000
2021-06-27  ID016  500.000000
2021-07-04  ID016  500.000000
2021-07-11  ID016  500.000000
2021-07-18  ID016  500.000000
2021-07-25  ID016  500.000000
2021-08-01  ID016  500.000000
2021-08-08  ID016  500.000000
2022-04-24  ID028  474.450000
2022-05-01  ID028  500.000000
2022-05-08  ID028  500.000000
2022-05-15  ID028  500.000000
2022-05-22  ID028  500.000000
2022-05-29  ID028  500.000000
2022-06-05  ID028  500.000000
2022-06-12  ID028  500.000000
2022-06-19  ID028  500.000000
2022-07-31  ID028  500.000000
2022-08-07  ID028  500.000000
2022-08-14  ID028  449.235507
2021-06-06  ID029  433.800000
2021-06-13  ID029  500.000000
2021-06-20  ID029  500.000000
2021-06-27  ID029  500.000000
2021-07-04  ID029  490.153846
2021-07-25  ID032  500.000000
2021-08-01  ID032  500.000000
202

In [40]:
df_weekly.isna().sum()

id     0
AQI    0
dtype: int64

In [41]:
df_weekly.shape

(2314, 2)

In [42]:
df_weekly.tail()

,id,AQI
DateTime,,
2022-09-18,ID035,71.848485
2022-09-25,ID035,59.061538
2022-10-02,ID035,56.682927
2022-10-09,ID035,52.505837
2022-10-16,ID035,59.309091


In [43]:
df_filtered = df_weekly[df_weekly['id'] != 'ID023']
df_filtered

,id,AQI
DateTime,,
2021-06-06,ID001,0.000000
2021-06-13,ID001,0.000000
2021-06-20,ID001,0.000000
2021-06-27,ID001,0.000000
2021-07-04,ID001,0.000000
...,...,...
2022-09-18,ID035,71.848485
2022-09-25,ID035,59.061538
2022-10-02,ID035,56.682927


In [44]:
df_filtered.id.unique()

array(['ID001', 'ID003', 'ID004', 'ID005', 'ID006', 'ID007', 'ID008',
       'ID009', 'ID010', 'ID011', 'ID012', 'ID013', 'ID014', 'ID015',
       'ID016', 'ID017', 'ID018', 'ID020', 'ID021', 'ID022', 'ID024',
       'ID025', 'ID026', 'ID027', 'ID028', 'ID029', 'ID030', 'ID031',
       'ID032', 'ID033', 'ID034', 'ID035'], dtype=object)

In [46]:
class TSA:
    def __init__(self,df,idcol,loc):
        self.df = df
        self.idcol = idcol
        self.loc = loc
        
        
    def zone_df(self):
        self.df = self.df[self.df[self.idcol]== self.loc]
        print(self.loc)
    
    def adf_test(self,valcol):
        """
        Pass in a time series and an optional title, returns an ADF report
        """
        print('Testing')
        result = adfuller(self.df[valcol].dropna(),autolag='AIC') # .dropna() handles differenced data
        print('result')
    
        labels = ['ADF test statistic','p-value','# lags used','# observations']
        out = pd.Series(result[0:4],index=labels)

        for key,val in result[4].items():
            out[f'critical value ({key})']=val
        
        print(out.to_string())          # .to_string() removes the line "dtype: float64"
    
        if result[1] <= 0.05:
            print("Strong evidence against the null hypothesis")
            print("Reject the null hypothesis")
            print("Data has no unit root and is stationary")
        else:
            print("Weak evidence against the null hypothesis")
            print("Fail to reject the null hypothesis")
            print("Data has a unit root and is non-stationary")
    def determine_ARIMA_order(self,valcol):
        stepwise_fit = auto_arima(self.df[valcol], start_p=0, start_q=0,
                          error_action='ignore',   # we don't want to know if an order does not work
                          suppress_warnings=True,  # we don't want convergence warnings
                          stepwise=True)           # set to stepwise
        best_order = stepwise_fit.get_params().get('order')
        print('The best order is '.format(best_order))
        return best_order
    def fit_model(self,valcol):
#         if len(self.df[valcol]) > 70:
        train = self.df[valcol][:len(self.df[valcol])-8]
        test = self.df[valcol][len(self.df[valcol])-8:len(self.df[valcol])-4]
        val = self.df[valcol][len(self.df[valcol])-4:]
#         else:
#             train = self.df[valcol][:len(self.df[valcol])-4]
#             test = self.df[valcol][len(self.df[valcol])-4:]
        start = len(train)
        end = len(train)+len(test)-1
#             print('train : {}'.format(train))
#             print('test : {}'.format(test))
        print('start : {}'.format(start))
        print('end : {}'.format(end))
        results = ARIMA(train,order=c1.determine_ARIMA_order(valcol)).fit()
        predictions = results.predict(start=start, end=end)
        error1 = mean_squared_error(test, predictions)
        error2 = rmse(test, predictions)
        print(f'MSE Error: {error1:11.10}')
        print(f'RMSE Error: {error2:11.10}')
            
    def full_data_model(self,valcol):
        results = ARIMA(self.df[valcol],order=c1.determine_ARIMA_order(valcol)).fit()
        
#         with open('TSA_AQI_{}.pkl'.format(self.loc),'wb')as f:
#             pickle.dump(results,f)
            
        order=c1.determine_ARIMA_order(valcol)
#         with open('Order_TSA_AQI_{}.pkl'.format(self.loc),'wb')as f:
#             pickle.dump(order,f)
            
#         if len(self.df[valcol]) > 70:
        fcast = results.predict(len(self.df), len(self.df)+3,typ='levels').round(2)
#         else:
#             fcast = results.predict(len(self.df), len(self.df)+3,typ='levels').round(2)
#         ax = self.df[valcol].plot(legend=True,figsize=(12,6))
#         fcast.plot(legend=True)
        print(order)
        print(fcast)
        DF = pd.DataFrame(self.df[valcol])
        DF['Type'] = 'Actual'
        DF_fcast = pd.DataFrame(fcast)
        DF_fcast = DF_fcast.rename(columns={'predicted_mean':'AQI'})
        DF_fcast['Type'] = 'Predicted'
        final_DF =  pd.concat([DF,DF_fcast], ignore_index=True)
        final_DF = final_DF.reset_index()
        final_DF = final_DF.rename(columns={'index':'Date'})
        print(final_DF)
        #final_DF.to_json('/Users/nithingopinath/Desktop/Bayesian Ways/AQI Deployment\{}.json'.format(item),orient='records')

id_list = list(df_filtered.id.unique())
for item in id_list:
    c1 = TSA(df_weekly,'id',item)
    c1.zone_df()
    c1.adf_test('AQI')
    c1.determine_ARIMA_order('AQI')
    c1.fit_model('AQI')
    c1.full_data_model('AQI')

ID001
Testing
result
ADF test statistic      -1.969205
p-value                  0.300237
# lags used              0.000000
# observations          71.000000
critical value (1%)     -3.526005
critical value (5%)     -2.903200
critical value (10%)    -2.588995
Weak evidence against the null hypothesis
Fail to reject the null hypothesis
Data has a unit root and is non-stationary
The best order is 
start : 64
end : 67
The best order is 
MSE Error: 71.87365297
RMSE Error: 8.477833035


/Users/shilpa.av/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-SUN will be used.
  self._init_dates(dates, freq)
/Users/shilpa.av/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-SUN will be used.
  self._init_dates(dates, freq)
/Users/shilpa.av/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-SUN will be used.
  self._init_dates(dates, freq)


The best order is 


/Users/shilpa.av/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-SUN will be used.
  self._init_dates(dates, freq)
/Users/shilpa.av/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-SUN will be used.
  self._init_dates(dates, freq)
/Users/shilpa.av/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-SUN will be used.
  self._init_dates(dates, freq)


The best order is 
(1, 0, 0)
2022-10-23    59.92
2022-10-30    60.76
2022-11-06    61.51
2022-11-13    62.18
Freq: W-SUN, Name: predicted_mean, dtype: float64
    Date    AQI       Type
0      0   0.00     Actual
1      1   0.00     Actual
2      2   0.00     Actual
3      3   0.00     Actual
4      4   0.00     Actual
..   ...    ...        ...
71    71  59.00     Actual
72    72  59.92  Predicted
73    73  60.76  Predicted
74    74  61.51  Predicted
75    75  62.18  Predicted

[76 rows x 3 columns]
ID003
Testing
result
ADF test statistic      -2.486959
p-value                  0.118680
# lags used              0.000000
# observations          71.000000
critical value (1%)     -3.526005
critical value (5%)     -2.903200
critical value (10%)    -2.588995
Weak evidence against the null hypothesis
Fail to reject the null hypothesis
Data has a unit root and is non-stationary


...

/Users/shilpa.av/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-SUN will be used.
  self._init_dates(dates, freq)
/Users/shilpa.av/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-SUN will be used.
  self._init_dates(dates, freq)
/Users/shilpa.av/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-SUN will be used.
  self._init_dates(dates, freq)


The best order is 


/Users/shilpa.av/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-SUN will be used.
  self._init_dates(dates, freq)
/Users/shilpa.av/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-SUN will be used.
  self._init_dates(dates, freq)
/Users/shilpa.av/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-SUN will be used.
  self._init_dates(dates, freq)


The best order is 
(1, 0, 0)
2022-10-23    55.90
2022-10-30    53.15
2022-11-06    50.94
2022-11-13    49.17
Freq: W-SUN, Name: predicted_mean, dtype: float64
    Date        AQI       Type
0      0   0.000000     Actual
1      1   0.000000     Actual
2      2   0.000000     Actual
3      3   0.000000     Actual
4      4   0.000000     Actual
..   ...        ...        ...
71    71  59.309091     Actual
72    72  55.900000  Predicted
73    73  53.150000  Predicted
74    74  50.940000  Predicted
75    75  49.170000  Predicted

[76 rows x 3 columns]


/Users/shilpa.av/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/statespace/representation.py:374: FutureWarning: Unknown keyword arguments: dict_keys(['typ']).Passing unknown keyword arguments will raise a TypeError beginning in version 0.15.
  warnings.warn(msg, FutureWarning)


In [10]:

#pickle_out = open("model.pkl","wb")
#pickle.dump(results, pickle_out)
#pickle_out.close()

In [11]:
#Arima_model = ARIMA(random_state=0).fit()
#Arima_model = ARIMA(train,order=c1.determine_ARIMA_order(valcol)).fit()
#pickle.dump(Arima_model, open("/Users/nithingopinath/Desktop/Bayesian Ways/AQI Deployment using fastapi/ArimaModel.pkl","wb"))

# SARIMA Model

In [47]:
class TSA_seasonal:
    def __init__(self, df,idcol,loc):
        self.df = df
        self.idcol = idcol
        self.loc = loc
    
    def zone_df(self):
        self.df = self.df[self.df[self.idcol]== self.loc]
        print(self.loc)
        
#     def find_D(self,valcol):
#          # estimate number of seasonal differences using a Canova-Hansen test
#         D = nsdiffs(self.df[valcol],m=12,
#             test='ch')  # -> 0
#         return D

    def adf_test(self, valcol):
        """
        Pass in a time series and an optional title, returns an ADF report
        """
        result = adfuller(self.df[valcol].dropna(), autolag='AIC')  # .dropna() handles differenced data

        labels = ['ADF test statistic', 'p-value', '# lags used', '# observations']
        out = pd.Series(result[0:4], index=labels)

        for key, val in result[4].items():
            out[f'critical value ({key})'] = val

        if result[1] <= 0.05:
            state = "Stationary"
        else:
            state = "Non-stationary"
        return state

    def determine_SARIMA_order(self, valcol):
#         D = self.find_D(valcol)
        stepwise_fit = auto_arima(self.df[valcol], seasonal=True, m=12,
                               start_p=0, start_q=0,
#                               start_P=0, start_Q=0,
#                                   D=D,
                                  error_action='ignore',  # we don't want to know if an order does not work
                                  suppress_warnings=True,  # we don't want convergence warnings
                                  stepwise=True)  # set to stepwise
        best_order = stepwise_fit.get_params().get('order')
        best_seasonal_order = stepwise_fit.get_params().get('seasonal_order')
        print('The best seasonal order is {}'.format(best_seasonal_order))
        print('The best order is {}'.format(best_order))
        return best_order, best_seasonal_order

    def fit_model(self, valcol):
    # Split the data into train, test, and validation sets
        train = self.df[valcol][:len(self.df[valcol]) - 8]
        test = self.df[valcol][len(self.df[valcol]) - 8:len(self.df[valcol]) - 4]
        val = self.df[valcol][len(self.df[valcol]) - 4:]

        # Determine the best SARIMA order
        best_order, best_seasonal_order = self.determine_SARIMA_order(valcol)

        # Fit the SARIMA model on the training data
        model = SARIMAX(train, order=best_order, seasonal_order=best_seasonal_order)
        results = model.fit()

        # Generate predictions for the test set
        predictions = results.predict(start=len(train), end=len(train) + len(test) - 1)

        # Generate predictions for the validation set
        predictions_val = results.predict(start=len(train) + len(test), end=len(train) + len(test) + len(val) - 1)

        # Calculate error metrics for the test set
        error1 = mean_squared_error(test, predictions)
        error2 = mean_squared_error(test, predictions, squared=False)  # RMSE
        error3 = mean_absolute_percentage_error(test, predictions)
        accuracy = (1 - error3) * 100
        print(f'MSE Error: {error1:11.10}')
        print(f'RMSE Error: {error2:11.10}')
        print(f'MAPE Error: {error3:11.10}')
        print(f'Accuracy: {accuracy:11.10}')
        return predictions_val

    def full_data_model(self, valcol):
        best_order, best_seasonal_order = self.determine_SARIMA_order(valcol)
        model = SARIMAX(self.df[valcol], order=best_order, seasonal_order=best_seasonal_order)
        results = model.fit()
        fcast = results.forecast(steps=4).round(2)  # Forecast 4 steps ahead
#         print(fcast)
#         fcast_index = pd.date_range(start=self.df.index[-1], periods=4 + 1, freq='M')[1:]  # Assuming monthly data
        DF_fcast = pd.DataFrame({valcol: fcast})
        DF_fcast['Type'] = 'Predicted'
        print(DF_fcast)
        DF = pd.DataFrame(self.df[valcol])
        DF['Type'] = 'Actual'
        # Concatenate original data and forecast data
        combined_DF = pd.concat([DF, DF_fcast])
        combined_DF = combined_DF.reset_index().rename(columns={'index':'Date'})
        print(combined_DF)
        DF_val = pd.DataFrame(c2.fit_model(valcol))
        DF_val = DF_val.reset_index()
        DF_val = DF_val.rename(columns={'index':'Date','predicted_mean':'Validation'})
#         combined_DF = combined_DF.rename(columns={'index':'Date'})
        print(DF_val)
        print(combined_DF)
#         final_DF =  final_DF.merge(DF_val, on='DateTime',how='outer')
        final_DF =  combined_DF.merge(DF_val, on='Date',how='outer')
        final_DF['Date'] = final_DF['Date'].astype('str')
#         print(combined_df)
        return final_DF

id_list = list(df_filtered.id.unique())
for item in id_list:
    c2 = TSA_seasonal(df_weekly,'id',item)
    c2.zone_df()
#     c2.find_D('AQI')
    c2.adf_test('AQI')
    c2.determine_SARIMA_order('AQI')
    c2.fit_model('AQI')
    c2.full_data_model('AQI')
    


ID001
The best seasonal order is (0, 0, 0, 12)
The best order is (1, 0, 0)
The best seasonal order is (0, 0, 0, 12)
The best order is (1, 0, 0)
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            2     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  4.97799D+00    |proj g|=  4.43893D-03

At iterate    5    f=  4.97694D+00    |proj g|=  1.44141D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    2      7      9      1     0     0   2.075D-07   4.977D+00
  F =   4.9769418454029566     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL 

/Users/shilpa.av/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-SUN will be used.
  self._init_dates(dates, freq)
/Users/shilpa.av/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-SUN will be used.
  self._init_dates(dates, freq)
 This problem is unconstrained.


The best seasonal order is (0, 0, 0, 12)
The best order is (1, 0, 0)
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            2     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  4.92377D+00    |proj g|=  3.87200D-03

At iterate    5    f=  4.92287D+00    |proj g|=  9.06207D-05

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    2      6      8      1     0     0   5.695D-06   4.923D+00
  F =   4.9228721544772371     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            
              AQI       Type
2022-10-23  55.79  Predicted
2022-

/Users/shilpa.av/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-SUN will be used.
  self._init_dates(dates, freq)
/Users/shilpa.av/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-SUN will be used.
  self._init_dates(dates, freq)
 This problem is unconstrained.


The best seasonal order is (0, 0, 0, 12)
The best order is (1, 0, 0)
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            2     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  4.97799D+00    |proj g|=  4.43893D-03

At iterate    5    f=  4.97694D+00    |proj g|=  1.44141D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    2      7      9      1     0     0   2.075D-07   4.977D+00
  F =   4.9769418454029566     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            
MSE Error: 220.0597266
RMSE Error: 14.83441022
MAPE Error: 0.34

...

/Users/shilpa.av/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-SUN will be used.
  self._init_dates(dates, freq)
/Users/shilpa.av/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-SUN will be used.
  self._init_dates(dates, freq)
 This problem is unconstrained.


The best seasonal order is (0, 0, 0, 12)
The best order is (1, 0, 0)
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            2     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  4.74857D+00    |proj g|=  5.35058D-03

At iterate    5    f=  4.74812D+00    |proj g|=  5.81516D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    2      8     10      1     0     0   4.191D-06   4.748D+00
  F =   4.7481097952953020     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            
              AQI       Type
2022-10-23  52.96  Predicted
2022-

/Users/shilpa.av/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-SUN will be used.
  self._init_dates(dates, freq)
/Users/shilpa.av/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-SUN will be used.
  self._init_dates(dates, freq)
 This problem is unconstrained.


The best seasonal order is (0, 0, 0, 12)
The best order is (1, 0, 0)
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            2     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  4.79525D+00    |proj g|=  6.25244D-03

At iterate    5    f=  4.79476D+00    |proj g|=  7.38751D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    2      7      9      1     0     0   4.338D-06   4.795D+00
  F =   4.7947421147814602     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            
MSE Error: 883.3735643
RMSE Error: 29.72160097
MAPE Error: 0.40

/Users/shilpa.av/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-SUN will be used.
  self._init_dates(dates, freq)
/Users/shilpa.av/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-SUN will be used.
  self._init_dates(dates, freq)
 This problem is unconstrained.
